In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


In [ ]:
# Define constants
image_height = 100
image_width = 100
num_channels = 3
num_classes = 4  # we have 4 classes
batch_size = 32
epochs = 10

In [ ]:
# Define the dataset directory
dataset_dir = "/content/drive/Dataset"

In [ ]:
# Create ImageDataGenerator for augmentation and splitting
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Split the data into 80% training and 20% validation
)


In [ ]:
# Generate training dataset
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use training subset
)

Found 373 images belonging to 4 classes.


In [ ]:
# Generate validation dataset
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use validation subset
)

Found 92 images belonging to 4 classes.


In [ ]:
class_indices = train_generator.class_indices
print("Class Indices:", class_indices)

Class Indices: {'Acne': 0, 'impetigo': 1, 'melasma': 2, 'rosacea': 3}


In [ ]:
# Create CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 16928)             0         
                                                                 
 dense (Dense)               (None, 64)                1

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/10
12/12 [==============================] - 32s 3s/step - loss: 0.9753 - accuracy: 0.6729 - val_loss: 0.8542 - val_accuracy: 0.6957
Epoch 2/10
12/12 [==============================] - 5s 410ms/step - loss: 0.8652 - accuracy: 0.6890 - val_loss: 0.7781 - val_accuracy: 0.7283
Epoch 3/10
12/12 [==============================] - 8s 653ms/step - loss: 0.7872 - accuracy: 0.7185 - val_loss: 0.7911 - val_accuracy: 0.7283
Epoch 4/10
12/12 [==============================] - 5s 393ms/step - loss: 0.7026 - accuracy: 0.7319 - val_loss: 0.7842 - val_accuracy: 0.7174
Epoch 5/10
12/12 [==============================] - 5s 406ms/step - loss: 0.6280 - accuracy: 0.7855 - val_loss: 0.8291 - val_accuracy: 0.6848
Epoch 6/10
12/12 [==============================] - 8s 596ms/step - loss: 0.5563 - accuracy: 0.8016 - val_loss: 0.8135 - val_accuracy: 0.7391
Epoch 7/10
12/12 [==============================] - 5s 417ms/step - loss: 0.5439 - accuracy: 0.8016 - val_loss: 0.9707 - val_accuracy: 0.6957
Epoch 8/

In [ ]:
# Evaluate the model on the validation data
validation_loss, validation_accuracy = model.evaluate(validation_generator)

print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)


3/3 [==============================] - 1s 184ms/step - loss: 1.0597 - accuracy: 0.6957
Validation Loss: 1.059656023979187
Validation Accuracy: 0.695652186870575


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess the image
img_path = "/content/rosacea/rosacea-0024__ProtectWyJQcm90ZWN0Il0_FocusFillWzI5NCwyMjIsInkiLDg1XQ.jpg"  # Replace with the path to your random image
img = image.load_img(img_path, target_size=(image_height, image_width))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalize pixel values

# Predict the class probabilities
predictions = model.predict(img_array)

# Get the predicted class label
predicted_class = np.argmax(predictions)

# Get class indices from the validation generator
class_indices = train_generator.class_indices  # You can use train_generator or validation_generator, depending on your choice

# Invert the dictionary to map indices to class names
class_names = {v: k for k, v in class_indices.items()}

# Get the predicted class name
predicted_class_name = class_names[predicted_class]

# Print the predicted class name
print("Predicted class:", predicted_class_name)


1/1 [==============================] - 0s 24ms/step
Predicted class: rosacea


In [ ]:
# Save the model to a file
model.save('/content/skin_disorder_classifier_EfficientNetB2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
